In [54]:
import numpy as np
import pandas as pd
import itertools as it

filename        = "Dataset_Covid_CE.csv"
k_values        = [2, 4, 8, 16]

def get_filename_out(k):
    return f"Dataset_Covid_CE_Anon_Sup_k{k}.csv"

def generate_csv(df, k):
    return df.to_csv(get_filename_out(k))

def is_k_anon(table, qt, k = 2):
    """
    para a tabela ser considerada k-anonima, todo elemento da tabela deve possuir pelo menos k outros elementos com a mesma combinaçao de semi-identificadores. ou seja, para k = 2 e qt = {Atr1, Atr2}, todo registro da tabela deve aparecer pelo menos 2 vezes com mesmo valor nos atributos "Atr1" e "Atr2".

    essa função agrupa os elementos da tabela pelos atributos semi-identificadores em qt e retorna False se algum grupo ocorre menos que k vezes, True se todo grupo ocorre ao menos k vezes.
    """

    groups = table.groupby(by=qt).size()
    for n_of_groups in groups:
        if n_of_groups < k: 
            return False
    return True

In [55]:
# informações do dataset original

# df = pd.read_csv(filename, nrows=500000)
# df.comorbidadeCardiovascularSivep.value_counts(dropna=False)


In [56]:
# limpeza

semi_id = [
    "municipioCaso",
    "sexoCaso",
    "dataNascimento",
    "resultadoFinalExame",
    "racaCor",
]

sensitive = [
    "comorbidadeCardiovascularSivep",
    "comorbidadeDiabetesSivep",
]

dtype = {
    "municipioCaso"                  : "str",
    "sexoCaso"                       : "str",
    "dataNascimento"                 : "str",
    "resultadoFinalExame"            : "str",
    "comorbidadeCardiovascularSivep" : "str",
    "comorbidadeDiabetesSivep"       : "str",
    "racaCor"                        : "str",
}

date_columns = [
    "dataNascimento",
]

date_parser = lambda x: pd.to_datetime(x, format="%Y-%m-%d", errors = 'coerce')

# pegar apenas os atributos desejados
# df = pd.read_csv(filename, nrows=100000, usecols=semi_id + sensitive, dtype=dtype, parse_dates=date_columns, date_parser=date_parser)
df = pd.read_csv(filename, usecols=semi_id + sensitive, dtype=dtype, parse_dates=date_columns, date_parser=date_parser)
df = df.dropna(how="all") # remover registros com todos os valores nulo
df = df.fillna("*") # padroniza valores nulos
df

,municipioCaso,sexoCaso,dataNascimento,resultadoFinalExame,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep,racaCor
19,SOBRAL,MASCULINO,2003-08-14 00:00:00,Negativo,*,*,Parda
20,PACAJUS,MASCULINO,1983-11-07 00:00:00,Negativo,*,*,Parda
21,HORIZONTE,FEMININO,1982-01-14 00:00:00,Negativo,*,*,*
25,FORTALEZA,MASCULINO,1992-03-12 00:00:00,Negativo,*,*,Parda
28,CAUCAIA,MASCULINO,1970-03-06 00:00:00,Negativo,*,*,Sem Informacao
...,...,...,...,...,...,...,...
1266272,FORTALEZA,FEMININO,1982-12-08 00:00:00,Positivo,*,*,Branca
1266273,FORTALEZA,FEMININO,1980-12-02 00:00:00,Negativo,*,*,Parda
1266274,FORTALEZA,FEMININO,1988-05-17 00:00:00,Provável,*,*,Branca
1266275,CRATEUS,FEMININO,1970-09-28 00:00:00,Negativo,*,*,Parda


In [57]:
is_k_anon(df, semi_id, k=2)

False

In [58]:
# 1. suprimir valores de cada semi-identificador que ocorrem < k vezes (pois não seria possível formar >= k grupos com eles)
def supress_unique_k_values(table, qt, k = 2):
    for col in qt:
        to_replace = []
        value_count = dict(table[col].value_counts(dropna=False))
        for value in value_count:
            if (value_count[value] < k):
                to_replace.append(value)
        table[col] = table[col].replace(to_replace, "*")
    return table

df1 = supress_unique_k_values(df, semi_id, k=2)
df1

In [116]:
# 2. gerar tabela k = 2

# remove os grupos k >= 2 já formados da tabela
qt_5 = df1[df1.duplicated(subset=semi_id, keep=False)]
df2 = df1.drop_duplicates(subset=semi_id, keep=False)

df_splices = [qt_5]

for n in [4, 3, 2, 1]:
    semi_id_combinations = it.combinations(semi_id, n)
    best_semi_id_combo = []
    duplicate_rows = []

    # para cada combinação de semi identificadores, checa quantos grupos k >= 2 formaria e pega a que gera mais
    for s in semi_id_combinations:
        rows = df2[df2.duplicated(subset=s, keep=False)]
        if len(duplicate_rows) < len(rows):
            duplicate_rows = rows.copy()
            best_semi_id_combo = s

    # pega o(s) atributo(s) que não aparece(m) na combinação e aplica supressão
    rm = list(set(semi_id) - set(best_semi_id_combo))
    for col in rm:
        duplicate_rows[col] = "*"

    # remove grupos recém formados da tabela original
    df_splices.append(duplicate_rows)
    df2 = df2.drop_duplicates(subset=best_semi_id_combo, keep=False)

df2.dataNascimento = "*"
df2.racaCor = "*"
df_splices.append(df2)

df2 = pd.concat(df_splices).sort_index()
df2

,municipioCaso,sexoCaso,dataNascimento,resultadoFinalExame,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep,racaCor
19,SOBRAL,MASCULINO,*,Negativo,*,*,Parda
20,PACAJUS,MASCULINO,1983-11-07 00:00:00,Negativo,*,*,Parda
21,HORIZONTE,FEMININO,*,Negativo,*,*,*
25,FORTALEZA,MASCULINO,1992-03-12 00:00:00,Negativo,*,*,Parda
28,CAUCAIA,MASCULINO,*,Negativo,*,*,Sem Informacao
...,...,...,...,...,...,...,...
1266272,FORTALEZA,FEMININO,*,Positivo,*,*,Branca
1266273,FORTALEZA,FEMININO,1980-12-02 00:00:00,Negativo,*,*,Parda
1266274,FORTALEZA,FEMININO,*,Provável,*,*,Branca
1266275,CRATEUS,FEMININO,1970-09-28 00:00:00,Negativo,*,*,Parda


In [118]:
is_k_anon(df2, semi_id, k=2)
# generate_csv(df2, k=2)

In [92]:
# 3. gerar tabela k = 4
# for name, group in df2.groupby(semi_id):
#     for index, row in group.iterrows():
#         print(index)

76
318889
318993
325428
326853
332940
336979
337676
338109
344571
345566
348127
350631
379033
385935
722951
723806
759540
1259339
41047
43539
53615
75058
150131
159763
160231
160554
160598
162825
162853
162904
163656
167224
231690
287555
1263917
387186
583749
583750
584573
135282
728210
117382
117386
124793
1044591
253897
1195057
454590
853555
561225
852903
404950
993854
285242
1134797
354623
354625
693242
1243611
149061
1226593
409393
409398
398913
598523
598524
194453
426517
627743
1127179
679831
679832
679833
52446
52448
607833
607834
607835
607836
642619
164219
191025
71054
205258
472822
478153
393478
1228239
227346
1205701
388745
1123917
225970
1129628
475309
491758
682731
1202192
181985
1242331
623233
623234
623235
145199
148877
464210
148876
391570
86698
86700
229856
229866
245327
647115
414354
994564
159205
421426
246112
1131205
432564
646797
395242
912692
281278
787676
159555
224588
226309
234013
235423
245197
266971
267782
270132
318361
377873
490467
1256734
102408
132162
257

KeyboardInterrupt: 

In [76]:
# lll = ["municipioCaso", "sexoCaso", "dataNascimento", "resultadoFinalExame", "racaCor"]
# groups = ndf.groupby(by=["municipioCaso", "sexoCaso", "dataNascimento", "resultadoFinalExame", "racaCor"]).size()
# print(groups)
# take_out = []
# for g in groups:
#     if len(g) >= k:
#         take_out.append(g)
# qt5 = ndf.loc[take_out]

# grouped = ndf.groupby(by=qt).filter(lambda x: len(x) >= k)
# grouped = ndf.groupby(by=qt).size().to_frame()
# a = []
# for name, group in grouped:
#     for index, row in group.iterrows():
#         a.append(row)
# r = grouped

# r = ndf[ndf.duplicated(subset=qt, keep=False)]
# return r
# return qt_3_max
# for index, row in table.iterrows():
#     query = ' & '.join([f'{col} == "{row[col]}"' for col in qt])
#     rows = ndf.query(query)
#     if (rows.shape[0] < k)
#         row["racaCor"] = "*"
        
# return ndf